# News Modeling

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [1]:
pip install pyLDAvis gensim spacy

INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
INFO: pip is still looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
  Using cached scikit_learn-1.5.2-cp39-cp39-win_amd64.whl.metadata (13 kB)
INFO: pip is still looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than u

### Import the libraries

In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import gensim
import spacy

### Load the dataset
#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

In [8]:
df = pd.read_json('newsgroups.json')
df.content

0        From: lerxst@wam.umd.edu (where's my thing)\nS...
1        From: guykuo@carson.u.washington.edu (Guy Kuo)...
2        From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3        From: jgreen@amber (Joe Green)\nSubject: Re: W...
4        From: jcm@head-cfa.harvard.edu (Jonathan McDow...
                               ...                        
11309    From: jim.zisfein@factory.com (Jim Zisfein) \n...
11310    From: ebodin@pearl.tufts.edu\nSubject: Screen ...
11311    From: westes@netcom.com (Will Estes)\nSubject:...
11312    From: steve@hcrlgw (Steven Collins)\nSubject: ...
11313    From: gunning@cco.caltech.edu (Kevin J. Gunnin...
Name: content, Length: 11314, dtype: object

### Preprocess the data

### Email Removal

In [9]:
df.content = df.content.str.replace('\S*@\S*\s?', '', regex=True)
df.content

0        From: (where's my thing)\nSubject: WHAT car is...
1        From: (Guy Kuo)\nSubject: SI Clock Poll - Fina...
2        From: (Thomas E Willis)\nSubject: PB questions...
3        From: (Joe Green)\nSubject: Re: Weitek P9000 ?...
4        From: (Jonathan McDowell)\nSubject: Re: Shuttl...
                               ...                        
11309    From: (Jim Zisfein) \nSubject: Re: Migraines a...
11310    From: Subject: Screen Death: Mac Plus/512\nLin...
11311    From: (Will Estes)\nSubject: Mounting CPU Cool...
11312    From: (Steven Collins)\nSubject: Re: Sphere fr...
11313    From: (Kevin J. Gunning)\nSubject: stolen CBR9...
Name: content, Length: 11314, dtype: object

### Newline Removal

In [10]:
df.content = df.content.str.replace('[\r\n]+', ' ', regex=True)
df.content

0        From: (where's my thing) Subject: WHAT car is ...
1        From: (Guy Kuo) Subject: SI Clock Poll - Final...
2        From: (Thomas E Willis) Subject: PB questions....
3        From: (Joe Green) Subject: Re: Weitek P9000 ? ...
4        From: (Jonathan McDowell) Subject: Re: Shuttle...
                               ...                        
11309    From: (Jim Zisfein)  Subject: Re: Migraines an...
11310    From: Subject: Screen Death: Mac Plus/512 Line...
11311    From: (Will Estes) Subject: Mounting CPU Coole...
11312    From: (Steven Collins) Subject: Re: Sphere fro...
11313    From: (Kevin J. Gunning) Subject: stolen CBR90...
Name: content, Length: 11314, dtype: object

### Single Quotes Removal

In [11]:
df.content = df.content.str.replace("'", '', regex=False)
df.content

0        From: (wheres my thing) Subject: WHAT car is t...
1        From: (Guy Kuo) Subject: SI Clock Poll - Final...
2        From: (Thomas E Willis) Subject: PB questions....
3        From: (Joe Green) Subject: Re: Weitek P9000 ? ...
4        From: (Jonathan McDowell) Subject: Re: Shuttle...
                               ...                        
11309    From: (Jim Zisfein)  Subject: Re: Migraines an...
11310    From: Subject: Screen Death: Mac Plus/512 Line...
11311    From: (Will Estes) Subject: Mounting CPU Coole...
11312    From: (Steven Collins) Subject: Re: Sphere fro...
11313    From: (Kevin J. Gunning) Subject: stolen CBR90...
Name: content, Length: 11314, dtype: object

### Tokenize
- Create **sent_to_words()** 
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [12]:
def sent_to_words(sentence):
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


In [14]:
df['content'] = df['content'].apply(lambda x: list(sent_to_words(x))[0])
df.content

0        [from, wheres, my, thing, subject, what, car, ...
1        [from, guy, kuo, subject, si, clock, poll, fin...
2        [from, thomas, willis, subject, pb, questions,...
3        [from, joe, green, subject, re, weitek, organi...
4        [from, jonathan, mcdowell, subject, re, shuttl...
                               ...                        
11309    [from, jim, zisfein, subject, re, migraines, a...
11310    [from, subject, screen, death, mac, plus, line...
11311    [from, will, estes, subject, mounting, cpu, co...
11312    [from, steven, collins, subject, re, sphere, f...
11313    [from, kevin, gunning, subject, stolen, cbr, r...
Name: content, Length: 11314, dtype: object

### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

In [15]:
en_stop = set(stopwords.words('english'))
stop_words_list = ['from', 'subject', 're', 'edu', 'use']
for word in stop_words_list:
	en_stop.add(word)

#### remove_stopwords( )

In [16]:
def remove_stopwords(sentence):
    stopwords_removed = []
    for token in sentence:
        if token not in en_stop:
            stopwords_removed.append(token)
    return stopwords_removed

In [17]:
df['content'] = df['content'].apply(lambda x: remove_stopwords(x))
df.content

0        [wheres, thing, car, nntp, posting, host, rac,...
1        [guy, kuo, si, clock, poll, final, call, summa...
2        [thomas, willis, pb, questions, organization, ...
3        [joe, green, weitek, organization, harris, com...
4        [jonathan, mcdowell, shuttle, launch, question...
                               ...                        
11309    [jim, zisfein, migraines, scans, distribution,...
11310    [screen, death, mac, plus, lines, organization...
11311    [estes, mounting, cpu, cooler, vertical, case,...
11312    [steven, collins, sphere, points, organization...
11313    [kevin, gunning, stolen, cbr, rr, organization...
Name: content, Length: 11314, dtype: object

### Get Bigrams with **gensim.models.Phrases** 

In [20]:
phrases = gensim.models.Phrases(df.content, min_count=1, threshold=100)
bigram = gensim.models.phrases.Phraser(phrases)
bigram_sentences = [bigram[sent] for sent in df.content]
print(bigram[df.content[0]])

['wheres', 'thing', 'car', 'nntp_posting', 'host_rac', 'wam_umd', 'organization', 'university', 'maryland_college', 'park', 'lines', 'wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'door', 'sports', 'car', 'looked', 'late_early', 'called_bricklin', 'doors', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone_tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'car', 'made', 'history', 'whatever', 'info_funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'neighborhood_lerxst']


#### Bigrams

In [21]:
def make_bigrams(sentence):
    return bigram[sentence]

In [22]:
df['content'] = df['content'].apply(lambda x: make_bigrams(x))
df.content

0        [wheres, thing, car, nntp_posting, host_rac, w...
1        [guy_kuo, si, clock, poll, final, call, summar...
2        [thomas, willis, pb, questions, organization, ...
3        [joe, green, weitek, organization, harris, com...
4        [jonathan_mcdowell, shuttle_launch, question, ...
                               ...                        
11309    [jim_zisfein, migraines_scans, distribution_wo...
11310    [screen, death, mac, plus, lines, organization...
11311    [estes, mounting_cpu, cooler, vertical, case, ...
11312    [steven_collins, sphere_points, organization, ...
11313    [kevin, gunning, stolen, cbr_rr, organization,...
Name: content, Length: 11314, dtype: object

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [23]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

c:\Users\Tommy\anaconda3\envs\machine_learning\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [24]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [25]:
data_lemmatized = lemmatization(df.content, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['s', 'thing', 'car', 'nntp_poste', 'wam_umd', 'organization', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late_early', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info_funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood_lerxst']]


### Create a Dictionary

In [26]:
dictionary = Dictionary(data_lemmatized)

### Filter low-frequency words

In [31]:
dictionary.filter_extremes(no_below=10, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data_lemmatized]

### Create Index 2 word dictionary

In [32]:
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [33]:
ldamodel = LdaModel(corpus, num_topics=20, id2word = id2word, passes=20)

### Print the Keyword in the 10 topics

In [34]:
pprint(ldamodel.print_topics(num_words=10))

[(0,
  '0.031*"team" + 0.024*"year" + 0.022*"game" + 0.019*"play" + 0.018*"player" '
  '+ 0.014*"good" + 0.013*"get" + 0.013*"well" + 0.012*"think" + 0.012*"go"'),
 (1,
  '0.053*"file" + 0.026*"entry" + 0.024*"program" + 0.021*"section" + '
  '0.012*"build" + 0.012*"ripem" + 0.011*"name" + 0.011*"author" + '
  '0.011*"rule" + 0.010*"number"'),
 (2,
  '0.048*"nntp_poste" + 0.042*"article" + 0.035*"host" + 0.023*"m" + '
  '0.022*"get" + 0.017*"know" + 0.015*"go" + 0.014*"think" + 0.012*"reply" + '
  '0.011*"look"'),
 (3,
  '0.018*"use" + 0.015*"server" + 0.014*"application" + 0.013*"set" + '
  '0.012*"version" + 0.012*"display" + 0.011*"code" + 0.011*"available" + '
  '0.010*"include" + 0.009*"program"'),
 (4,
  '0.015*"say" + 0.013*"people" + 0.011*"believe" + 0.009*"make" + '
  '0.009*"think" + 0.008*"know" + 0.007*"question" + 0.006*"many" + '
  '0.006*"mean" + 0.006*"also"'),
 (5,
  '0.019*"people" + 0.018*"war" + 0.017*"armenian" + 0.014*"government" + '
  '0.013*"israeli" + 0.012*"

## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

In [35]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))


Perplexity:  -7.037588727666272


### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [36]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5372632969941484


### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [37]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.104114 -0.145066       1        1  17.186323
4     -0.096399 -0.070814       2        1  15.672652
18    -0.127539  0.030457       3        1   6.145902
8      0.005422  0.160994       4        1   5.630459
2     -0.093133 -0.155298       5        1   5.355399
14    -0.067413  0.080877       6        1   5.150850
11    -0.028084  0.202740       7        1   5.016295
19     0.488652  0.076275       8        1   4.968244
0     -0.040718 -0.183341       9        1   4.826826
16    -0.009920 -0.045765      10        1   4.585390
3     -0.069408  0.220178      11        1   4.434449
10    -0.077470  0.123592      12        1   3.675170
7     -0.105998  0.043069      13        1   3.575175
5      0.045415 -0.046005      14        1   3.235310
13    -0.098570 -0.115472      15        1   3.078921
9      0.034197  0.001654      16        1   2.323244
1     -0.014660  0.129493      17        1   1.980301
12     0.048981 -0.061930      18        1   1.324658
15     0.086144 -0.092087      19        1   0.940536
6      0.224615 -0.153551      20        1   0.893899, topic_info=            Term          Freq         Total Category  logprob  loglift
2652          ax  46008.000000  46008.000000  Default  30.0000  30.0000
160      article   6657.000000   6657.000000  Default  29.0000  29.0000
16    nntp_poste   4610.000000   4610.000000  Default  28.0000  28.0000
965         game   1680.000000   1680.000000  Default  27.0000  27.0000
281         file   2355.000000   2355.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1774       flame     77.678088    339.864802  Topic20  -4.6879   3.2414
98            go    168.427259   5605.753548  Topic20  -3.9140   1.2123
126         play     92.559684   1101.337852  Topic20  -4.5126   2.2409
347          win     76.438992    823.811051  Topic20  -4.7040   2.3399
1341       score     62.979855    265.162845  Topic20  -4.8976   3.2798

[1402 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2651      8  0.097663             _
2651     16  0.899237             _
4820      9  0.985194           aaa
5282     18  0.973296            ab
3988     19  0.966622  abc_coverage
...     ...       ...           ...
31       17  0.049464          year
31       18  0.041654          year
31       19  0.002314          year
1493     14  0.076740          zone
1493     19  0.920878          zone

[4206 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 5, 19, 9, 3, 15, 12, 20, 1, 17, 4, 11, 8, 6, 14, 10, 2, 13, 16, 7])